In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import murmurhash3_32
from random import randint
import torch

In [2]:
data_path = "datasets/URL_data.csv"
data = pd.read_csv(data_path)
negative_sample = data.loc[(data['label'] == -1)]
positive_sample = data.loc[(data['label'] == 1)]
url_negative = negative_sample['url']
url = positive_sample['url']
n = len(url)
train_negative = negative_sample.sample(frac = 0.3)

In [3]:
def hashfunc(m):
    ss = randint(1, 99999999)
    def hash_m(x):
        return murmurhash3_32(x,seed=ss)%m
    return hash_m

In [ ]:
###CPU
class Ada_BloomFilter():
    def __init__(self, n, hash_len, k_max):
        self.n = n
        self.hash_len = int(hash_len)
        self.h = []
        for i in range(int(k_max)):
            self.h.append(hashfunc(self.hash_len))
        self.table = np.zeros(self.hash_len, dtype=int)
    def insert(self, key, k):
        for j in range(int(k)):
            t = self.h[j](key)
            self.table[t] = 1
    def test(self, key, k):
        test_result = 0
        match = 0
        for j in range(int(k)):
            t = self.h[j](key)
            match += 1*(self.table[t] == 1)
        if match == k:
            test_result = 1
        return test_result



def R_size(count_key, count_nonkey, R0):
    R = [0]*len(count_key)
    R[0] = R0
    for k in range(1, len(count_key)):
        R[k] = max(int(count_key[k] * (np.log(count_nonkey[0]/count_nonkey[k])/np.log(0.618) + R[0]/count_key[0])), 1)
    return R


def Find_Optimal_Parameters(c_min, c_max, num_group_min, num_group_max, R_sum, train_negative, positive_sample):
    c_set = np.arange(c_min, c_max+10**(-6), 0.1)
    FP_opt = train_negative.shape[0]

    k_min = 0
    for k_max in range(num_group_min, num_group_max+1):
        for c in c_set:
            tau = sum(c ** np.arange(0, k_max - k_min + 1, 1))
            n = positive_sample.shape[0]
            hash_len = R_sum
            bloom_filter = Ada_BloomFilter(n, hash_len, k_max)
            thresholds = np.zeros(k_max - k_min + 1)
            thresholds[-1] = 1.1
            num_negative = sum(train_negative['score'] <= thresholds[-1])
            num_piece = int(num_negative / tau) + 1
            score = train_negative.loc[(train_negative['score'] <= thresholds[-1]), 'score']
            score = np.sort(score)
            for k in range(k_min, k_max):
                i = k - k_min
                score_1 = score[score < thresholds[-(i + 1)]]
                if int(num_piece * c ** i) < len(score_1):
                    thresholds[-(i + 2)] = score_1[-int(num_piece * c ** i)]

            url = positive_sample['url']
            score = positive_sample['score']

            for score_s, url_s in zip(score, url):
                ix = min(np.where(score_s < thresholds)[0])
                k = k_max - ix
                bloom_filter.insert(url_s, k)
            ML_positive = train_negative.loc[(train_negative['score'] >= thresholds[-2]), 'url']
            url_negative = train_negative.loc[(train_negative['score'] < thresholds[-2]), 'url']
            score_negative = train_negative.loc[(train_negative['score'] < thresholds[-2]), 'score']

            test_result = np.zeros(len(url_negative))
            ss = 0
            for score_s, url_s in zip(score_negative, url_negative):
                ix = min(np.where(score_s < thresholds)[0])
                # thres = thresholds[ix]
                k = k_max - ix
                test_result[ss] = bloom_filter.test(url_s, k)
                ss += 1
            FP_items = sum(test_result) + len(ML_positive)
            print('False positive items: %d, Number of groups: %d, c = %f' %(FP_items, k_max, round(c, 2)))

            if FP_opt > FP_items:
                FP_opt = FP_items
                bloom_filter_opt = bloom_filter
                thresholds_opt = thresholds
                k_max_opt = k_max

    # print('Optimal FPs: %f, Optimal c: %f, Optimal num_group: %d' % (FP_opt, c_opt, num_group_opt))
    return bloom_filter_opt, thresholds_opt, k_max_opt


In [ ]:
c_min = 1.8
c_max = 2.1
num_group_min = 8
num_group_max = 12
R_sum = 200000

bloom_filter_opt, thresholds_opt, k_max_opt = Find_Optimal_Parameters(c_min, c_max, num_group_min, num_group_max, R_sum, train_negative, positive_sample)

In [6]:
###GPU
def hashfunc(m):
    ss = randint(1, 99999999)
    def hash_m(x):
        # Assuming x is a string, convert to bytes for hashing
        return murmurhash.mrmr.hash(x.encode('utf-8'), seed=ss) % m
    return hash_m
# def hashfunc(m):
#     ss = randint(1, 99999999)
#     def hash_m(x):
#         return murmurhash3_32(x,seed=ss)%m
#     return hash_m


class Ada_BloomFilter():
    def __init__(self, n, hash_len, k_max):
        self.n = n
        self.hash_len = int(hash_len)
        self.h = [hashfunc(self.hash_len) for _ in range(k_max)]
        # Initialize the table on GPU
        self.table = torch.zeros(self.hash_len, dtype=torch.int32, device='cuda')
    
    def insert(self, keys):
        positions = torch.zeros((len(keys), len(self.h)), dtype=torch.long, device='cuda')
        for i, key in enumerate(keys):
            for j, hash_func in enumerate(self.h):
                positions[i, j] = hash_func(key)
        # Flatten and remove duplicates before updating the table to prevent race conditions
        positions = positions.view(-1).unique()
        self.table[positions] = 1
    
    def test(self, keys):
        results = torch.ones(len(keys), dtype=torch.int32, device='cuda')
        for i, key in enumerate(keys):
            for j, hash_func in enumerate(self.h):
                if not self.table[hash_func(key)]:
                    results[i] = 0
                    break
        return results

def Find_Optimal_Parameters(c_min, c_max, num_group_min, num_group_max, R_sum, train_negative, positive_sample):
    c_set = torch.arange(c_min, c_max + 10**(-6), 0.1, device='cuda')
    FP_opt = train_negative.shape[0]

    for k_max in range(num_group_min, num_group_max + 1):
        for c in c_set:
            tau = torch.sum(c ** torch.arange(0, k_max + 1, device='cuda'))
            n = positive_sample.shape[0]
            hash_len = R_sum
            bloom_filter = Ada_BloomFilter(n, hash_len, k_max)
            thresholds = torch.zeros(k_max + 1, device='cuda')
            thresholds[-1] = 1.1  # Define the initial threshold
            num_negative = (train_negative['score'] <= thresholds[-1].item()).sum()
            num_piece = int(num_negative / tau.item()) + 1
            score = train_negative[train_negative['score'] <= thresholds[-1].item()]['score'].values
            score = torch.tensor(np.sort(score), device='cuda')
            for k in range(k_max):
                i = k
                score_1 = score[score < thresholds[-(i + 1)].item()]
                if int(num_piece * c.item() ** i) < len(score_1):
                    thresholds[-(i + 2)] = score_1[-int(num_piece * c.item() ** i)]

            keys = [url_s for score_s, url_s in zip(positive_sample['score'].values, positive_sample['url']) if score_s < thresholds[0].item()]
            bloom_filter.insert(keys)

            ML_positive = train_negative.loc[(train_negative['score'] >= thresholds[-2]), 'url']
            url_negative = train_negative[train_negative['score'] < thresholds[-2].item()]['url']
            score_negative = train_negative[train_negative['score'] < thresholds[-2].item()]['score'].values
            

            test_keys = [url_s for score_s, url_s in zip(score_negative, url_negative) if score_s < thresholds[0].item()]
            test_results = bloom_filter.test(test_keys)
            FP_items = test_results.sum().item()+ len(ML_positive)
            print(f'False positive items: {FP_items}, Number of groups: {k_max}, c = {round(c.item(), 2)}')

            if FP_opt > FP_items:
                FP_opt = FP_items
                bloom_filter_opt = bloom_filter
                thresholds_opt = thresholds
                k_max_opt = k_max

    return bloom_filter_opt, thresholds_opt, k_max_opt


In [7]:
c_min = 1.8
c_max = 2.1
num_group_min = 8
num_group_max = 12
R_sum = 200000

bloom_filter_opt, thresholds_opt, k_max_opt = Find_Optimal_Parameters(c_min, c_max, num_group_min, num_group_max, R_sum, train_negative, positive_sample)

AssertionError: Torch not compiled with CUDA enabled

In [11]:
import torch
print(torch.cuda.is_available())

False


In [9]:
import torch
torch.zeros(1).cuda()

AssertionError: Torch not compiled with CUDA enabled